# Progetto Gruppo 2
## Segmentazione di Cellule del sangue

**Autori:** Fortunato Michela, Iodice Luisa, Minervini Nicolas, Modano Armando

# Fase preliminare
Installiamo easy-cv-dataset che ci servirà

In [ ]:
!pip install --upgrade git+https://github.com/davin11/easy-cv-dataset keras-hub

Scarichiamo ed estraiamo il dataset

In [ ]:
!wget --user=corso --password=corso2025f https://www.grip.unina.it/download/corso/bbbc041seg-DatasetNinja.tar
!tar --skip-old-files -xf bbbc041seg-DatasetNinja.tar

Scarichiamo lo script "create-msk.py" per la creazione delle maschere<br>
Per evitare di dover importate manualmente ogni volta lo script "create_msk.py" è stato caricato su un GitHub personale e si scarica direttamente da lì<br>
Creiamo anche le maschere

In [ ]:
SITE = "https://raw.githubusercontent.com/Nicolas1617/university_projects/refs/heads/main/VSR%20project%20work"
!wget -nc {SITE}/create_msk.py
!python create_msk.py

# Import di base
Come prima cosa puliamo le variabili e importiamo le librerie che sicuramente ci serviranno, altre librerie le importeremo man mano all'occorrenza

In [ ]:
%reset -f
import numpy as np
import matplotlib.pyplot as plt
import skimage.io as io
import keras
import keras_hub
import easy_cv_dataset as ds

# Creazione delle tabelle di train e test
Di seguito creiamo le tabelle per i dati di train e di test

In [ ]:
from glob import glob
import pandas
list_train_img_path = [] #creiamo una lista vuota
list_train_msk_path = glob('train/msk/*.png')
for name in list_train_msk_path:
    name=name[:-4] #il nome della maschera ha 4 caratteri in più per questo ci creiamo un nome con 4 caratteri in meno
    name=name.replace('/msk/','/img/')
    list_train_img_path.append(name)

tab_train = pandas.DataFrame({'image': list_train_img_path,
                        'segmentation_mask': list_train_msk_path})
tab_train.to_csv('train_table.csv')

Abbiamo visto che le immagini non erano tutte .png ma c'erano anche alcune .jpg, per questo motivo abbiamo creato il vettore **list_img_path** a partire dal vettore **list_msk_path**.
Per farlo abbiamo confrontato i nomi delle immagini e delle maschere e abbiamo visto che i nomi delle maschere avevano 4 caratteri in più per questo motivo il vettore name prende tutti i caratteri tranne gli ultimi 4.
Abbiamo seguito questa procedura e non preso anche i file .jpg dalla cartella img perché altrimenti non avremmo avuto la certezza che maschera e immagine corrispondessero
Dato che siamo partiti dalla cartella delle maschere oltre ad accorciare il nome abbiamo dovuto sostituire la stringa **/msk/** con la stringa **/img/**

In [ ]:
# abbiamo stampato queste cose per capire come mai desse errore sulla lunghezza
#print(len(list_img_path))
#print(len(list_msk_path))
#print(list_msk_path[0])
#print(list_img_path[0])

In [ ]:
list_test_img_path = []
list_test_msk_path = glob('test/msk/*.png')
for name in list_test_msk_path:
    name=name[:-4]
    name=name.replace('/msk/','/img/')
    list_test_img_path.append(name)

tab_test = pandas.DataFrame({'image': list_test_img_path,
                        'segmentation_mask': list_test_msk_path})
tab_test.to_csv('test_table.csv')

# Creazione validation set
Di seguito creiamo il validation set che sarà comporsto da 159 elementi presi in maniera causale dal train set

In [ ]:
from sklearn.model_selection import train_test_split

tab_tran, tab_valid = train_test_split(tab_train, test_size=159, random_state=34)
tab_valid.to_csv('valid_table.csv')

# [Augmentation](https://keras.io/api/layers/preprocessing_layers/image_augmentation/)
Abbiamo deciso di fare operazioni random di variazione di luminosità, di rotazione e flip.<br>
La variazione di luminosità va in un range -20%,+20%<br>
La rotazione va da -5% di 360° a +5% di 360°<br>
Il flip è fatto in orizzontale

In [ ]:
#augmentation
from keras.layers import Pipeline, RandomBrightness, RandomRotation, RandomFlip
augmenter = Pipeline(layers=[
    RandomBrightness(factor=(-0.2, 0.2), value_range=(0, 255)),
    RandomRotation((-0.05,0.05)),
    RandomFlip("horizontal")])

# Definizione del dataset
Definiamo batch size, dimensione dell'immagine e dataset per train, validation e test

In [ ]:
batch_size = 8
image_size = 512
class_names = ["Background", "Foreground"]
num_classes = 2
from keras.layers import Resizing
pre_batching_processing = Resizing(image_size, image_size)
post_batching_processing = augmenter

print('test-set')
test_ds = ds.image_segmentation_dataset_from_dataframe(
    'test_table.csv',
    class_mode='categorical', class_names=class_names,
    pre_batching_processing=pre_batching_processing,
    shuffle=False, batch_size=batch_size) #nel test set non ha senso fare shuffle, ha senso solo nel training set

print('trainig-set')
train_ds = ds.image_segmentation_dataset_from_dataframe(
    'train_table.csv',
    class_mode='categorical', class_names=class_names,
    pre_batching_processing=pre_batching_processing,
    shuffle=True , batch_size=batch_size,
    post_batching_processing=post_batching_processing)

print('validation-set')
valid_ds = ds.image_segmentation_dataset_from_dataframe(
    'valid_table.csv',
    class_mode='categorical', class_names=class_names,
    pre_batching_processing=pre_batching_processing,
    shuffle=False, batch_size=batch_size)

# Visualizzazione
Di seguito visualizziamo un batch di immagini con la relativa mappa di segmentazione

In [ ]:
from easy_cv_dataset.visualization import plot_segmentation_mask_gallery

for images, segms in test_ds.take(1): # il for scorre solo sul primo batch
    plot_segmentation_mask_gallery( # funzione per visualizzare image and box
    images, y_true = segms, num_classes = num_classes
)

# Importazione modello pre-addestrato
[mit_b0_ade20k_512](https://www.kaggle.com/models/keras/mit/keras/mit_b0_ade20k_512/4)

In [ ]:
from keras_hub.models import MiTBackbone, SegFormerBackbone
from keras_hub.models import SegFormerImageSegmenter
from keras_hub.models import ImageSegmenterPreprocessor as pre

preprocessor = pre.from_preset("segformer_b0_ade20k_512")
encoder = MiTBackbone.from_preset("mit_b0_ade20k_512",
    load_weights=True,
    image_shape=(image_size, image_size, 3),
)
backbone = SegFormerBackbone(
    image_encoder=encoder,
    projection_filters=256,
    )
model = SegFormerImageSegmenter(
    preprocessor=preprocessor,
    backbone=backbone,
    num_classes=num_classes,
    activation=None #abbiamo messo None perché in seguito mettiamo logit=True
    )
model.summary()

# Train Backbone
Conviene non bloccare il backbone ma riaddestrarlo perché il modello è pre addestrato su un dataset molto diverso dal nostro.<br>
Abbiamo anche effettuato una prova con backbone bloccato e abbiamo visto un valore di circa 0.78 con backbone bloccato a fronte di un valore di circa 0.92 con backbone riaddestrato

In [ ]:
#model.backbone.trainable = False
#model.summary()
#abbiamo deciso di addestrare anche il backbone come test e abbiamo visto che siamo passati da 0.78 a 0.91

# Learning Rate variabile
Ha senso usare uno schedule del learning rate in modo da avere una rete che segue più step
- **Step 1** learning rate alto per un adattamento rapido
- **Step 2** learning rate più basso per stabilizzazione
- **Step 3** learning rate ancora più basso per finalizzazione

In [ ]:
base_lr = 0.001 #definiamo un learning rate (lr) di partenza
from keras import optimizers

lr_decay = optimizers.schedules.PiecewiseConstantDecay( #schedulatore del lr, serve per ridurre il lr secondo uno scheduling
    boundaries=[5*len(train_ds), 10*len(train_ds)], #qui diciamo che deve ridurre dopo 5 epoche e dopo 10 epoche
    values=[base_lr, 0.1 * base_lr, 0.01 * base_lr], #qui diciamo di quanto deve ridurre il lr
)
optimizer = optimizers.Nadam(
    learning_rate=lr_decay, global_clipnorm=10.0
)

# [Early Stop](https://keras.io/api/callbacks/early_stopping/)
L'eraly stop è una tecnica che permette di fermare l'addestramento in base ad alcuni criteri stabiliti:
- **monitor** indica il valore da monitorare, nel nostro caso abbiamo deciso di controllare la loss sulla validation
- **min_delta** indica la variazione minima da rispettare, nel nostro caso abbiamo detto che ci deve essere una variazione di almeno 0.001
- **patience** indica il numero di epoche su cui controllare il valore di min_delta
- **mode** indica che cosa stiamo facendo con il valore da monitorare, noi abbiamo indicato che vogliamo minimizzarlo
- **restore_best_weights** ci chiede se vogliamo tornare o meno al peso migliore ottenuto durante il training
- **start_from_epoch** chiede da quale epoca far iniziare il monitoraggio

Il fatto di avere impostato min come mode fa si che la variazione debba essere sempre verso la minimizzazione, altimenti anche un peggioramento con un delta di almeno 0.001 andrebbe bene, cosa che non vogliamo.<br>
Abbiamo deciso di impostare un'epoca di inizio perché abbiamo notato che altrimenti ci fermava troppo presto dato che con lr grande si verificavano delle variazioni di loss verso l'alto che ci facevano fermare l'addestramento troppo presto e in più perché non ci serve tanto per evitare overfitting ma per trovare un numero di epoche adatto per fermare l'addestramento quando non ci sono più miglioramenti per questo abbiamo deciso di far partire l'early stop quando stiamo addestrando con il lr più piccolo.<br>
In più abbiamo deciso di mettere ripristinare il peso migliore perché se l'addestramento si ferma per mancanza di miglioramenti il peso migliore sarà comunque quello dell'ultima epoca, ma se l'addestramento si ferma perché la loss stava peggiorando non conviene più prendere il peso dell'ultima epoca ma quello della migliore del training.

Altro vantaggio dell'Early Stop è quello di poter impostare un numero di epoche abbastanza alto in quanto l'addestramento si dovrebbe fermare prima, quindi quello di addestrare sul maggior numero di epoche possibili

In [ ]:
#definiamo l'early stopping
from keras.callbacks import EarlyStopping

early_stop = EarlyStopping(
    monitor='val_loss',         #guardo la loss di validazione
    min_delta=0.001,            #serve un miglioramento almeno di 0.001
    patience=3,                 #se per 3 epoche non migliora, mi fermo
    mode='min',                 #voglio minimizzare la loss
    restore_best_weights=True,  #torno ai pesi migliori
    start_from_epoch=10         #epoca dalla quale partire con il monitoraggio
)

# Loss function
Definiamo la nostra loss function<br>
Si può anche usare la Cross Entropy e non la Focal, i risultati non cambiano di molto

In [ ]:
from keras.losses import CategoricalFocalCrossentropy
from keras.optimizers import Nadam
from keras.metrics import MeanIoU
model.compile(
    loss=CategoricalFocalCrossentropy(from_logits=True), #mettiamo True perché non abbiamo messo la softmax prima, questa scelta è migliore per questioni di arrotondamento sul calcolatore
    optimizer=optimizer, #nadam è una variante della discesa lungo il gradiente stocastico
    metrics=[MeanIoU(num_classes=2, sparse_y_true=False, sparse_y_pred=False),], #non usiamo l'accuracy per quanto detto in teoria
)

# Addestramento
Di seguito addestriamo la rete specificando il train set, il validation set, il numero di epoche ed altri fattori tipo il verbose che permette di visualizzare una barra di progresso e la callback che ci permette di richiamare il nostro earlystop.<br>
Si è deciso di impostare un numero di epoche pari a 30 ma si poteva mettere anche più alto, tanto la presenza dell'early stop ferma l'addestramento prima<br>
Alla rete è stato assegnato un nome che ci tornarà utile nel passo successivo per poter disegnare i grafici<br>
Dopo aver addestrato salviamo i pesi in un file

In [ ]:
gruppo_2 = model.fit(train_ds, epochs=30, validation_data=valid_ds, verbose=True, callbacks=[early_stop])
model.save_weights('gruppo_2.weights.h5')

# Visualizzazione grafica andamento loss
Di seguito effettuiamo una visualizzazione grafica di come sta andando la loss e la loss sulla validation in modo da avere ad impatto grafico un'idea di quello che sta succedendo

In [ ]:
epoche = len(gruppo_2.history['loss']) #calcolo il numero di epoche come numero di valori di loss

plt.plot(gruppo_2.history['loss'], label='Training loss') #plotta la curva della training loss
plt.plot(gruppo_2.history['val_loss'], label='Validation loss') #plotta la curva della validation loss
plt.xticks(np.arange(epoche)) #setta l'asse x con i valori del numero di epoche
plt.xlabel('epochs') #rinomina l'asse x
plt.ylabel('loss value') #rinomina l'asse y
plt.title('Training vs Validation loss') #da un titolo al grafico
plt.legend() #crea una legenda
plt.show()

Visualizziamo anche l'andamento del valore di Mean IoU sia su train set sia su validation

In [ ]:
plt.plot(gruppo_2.history['mean_io_u'], label='Mean_io_u')
plt.plot(gruppo_2.history['val_mean_io_u'], label='Validation Mean_io_u')
plt.xticks(np.arange(epoche))
plt.xlabel('epochs')
plt.ylabel('Mean IoU value')
plt.title('Training vs Validation Mean IoU')
plt.legend()
plt.show()

# Test rete
Di seguito testiamo la rete e stampiamo le metriche

In [ ]:
metrics = model.evaluate(test_ds, return_dict=True, verbose=True)
print(metrics)

# Visualizzazion del risultato
Come avevamo fatto in precedenza anche qui visualizziamo un batch, in questo caso sarà presente anche la maschera predetta per poter fare un confronto visivo

In [ ]:
#visualizziamo il risultato
from easy_cv_dataset.visualization import plot_segmentation_mask_gallery
for images, segms in test_ds.take(1): #prendiamo il primo batch del testset
    pred = model.predict(images) #lanciamo la rete sul singolo batch con il comando predict
    plot_segmentation_mask_gallery(
        images, y_true=segms, y_pred=pred, num_classes=num_classes
    )
print(pred.shape)

# [Conteggio cellule](https://scikit-image.org/docs/0.25.x/api/skimage.measure.html#skimage.measure.label)
Di seguito contiamo le cellule presenti tramite la funzione label della libreria skimage.measure<br>
Measure restituisce due cose, una matrice che è la maschera e il numero contato

In [ ]:
from skimage.measure import label
index = 2
mask_pred, num_pred = label(pred[index,:,:,1]>0, background=None, return_num=True, connectivity=None)
mask_true, num_true = label(segms[index,:,:,1]>0, background=None, return_num=True, connectivity=None)
print('Cellule reali contate: ',num_true)
print('Cellule predette contate: ',num_pred)
plt.figure(figsize=(10,6))
plt.subplot(1,2,1)
plt.imshow(mask_true, cmap='jet')
plt.title('Maschera reale')
plt.axis('off')
plt.subplot(1,2,2)
plt.imshow(mask_pred, cmap='jet')
plt.title('Maschera predetta')
plt.axis('off')
plt.show()

# [Identificazione cellule](https://scikit-image.org/docs/0.25.x/auto_examples/segmentation/plot_label.html)
mpathes è una libreria che serve per disegnare forme, nel nostro caso rettangoli<br>
label_image serve per etichettare le regioni<br>
image_label_overlay colora ogni regione con un colore diverso e li sovrappone a mask_pred<br>
Il ciclo for serve per avere una lista di oggetti per ogni regione<br>
Vengono prese solo regioni di dimensione >= a 20<br>
Si danno le coordinate del rettangolo da fare con rigion.bbox<br>
rect disegna il rettangolo<br>
Alla fine si nascondono gli assi, si sistemano i margini e si visualizza

In [ ]:
import matplotlib.patches as mpatches

from skimage.filters import threshold_otsu
from skimage.segmentation import clear_border
from skimage.measure import label, regionprops
from skimage.morphology import closing, footprint_rectangle
from skimage.color import label2rgb

label_image = label(mask_pred)

image_label_overlay = label2rgb(label_image, image=mask_pred, bg_label=0)

fig, ax = plt.subplots(figsize=(10, 6))
ax.imshow(image_label_overlay)

for region in regionprops(label_image):
    if region.area >= 20:
        minr, minc, maxr, maxc = region.bbox
        rect = mpatches.Rectangle(
            (minc, minr),
            maxc - minc,
            maxr - minr,
            fill=False,
            edgecolor='red',
            linewidth=2,
        )
        ax.add_patch(rect)

ax.set_axis_off()
plt.tight_layout()
plt.show()